# Flujo de Trabajo para Procesamiento de Datos MODFLOW

Este notebook presenta un flujo de trabajo completo para el pre-procesamiento y post-procesamiento de datos de niveles observados y simulados obtenidos de modelos de aguas subterráneas MODFLOW.

## Estructura del Proyecto
- `data/raw/`: Datos originales (archivos .hds, .cbb, .obs)
- `data/processed/`: Datos procesados y limpiados
- `data/output/`: Resultados y análisis finales
- `scripts/`: Funciones auxiliares
- `figures/`: Gráficos y visualizaciones
- `reports/`: Reportes generados


In [ ]:
# Importar librerías necesarias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import os
import glob
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Configurar estilo de gráficos
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Configurar directorios del proyecto
PROJECT_ROOT = Path.cwd().parent
DATA_RAW = PROJECT_ROOT / "data" / "raw"
DATA_PROCESSED = PROJECT_ROOT / "data" / "processed"
DATA_OUTPUT = PROJECT_ROOT / "data" / "output"
FIGURES_DIR = PROJECT_ROOT / "figures"
REPORTS_DIR = PROJECT_ROOT / "reports"

# Crear directorios si no existen
for directory in [DATA_RAW, DATA_PROCESSED, DATA_OUTPUT, FIGURES_DIR, REPORTS_DIR]:
    directory.mkdir(parents=True, exist_ok=True)

print("✅ Librerías importadas y directorios configurados correctamente")
print(f"📁 Directorio del proyecto: {PROJECT_ROOT}")
print(f"📊 Datos originales: {DATA_RAW}")
print(f"🔧 Datos procesados: {DATA_PROCESSED}")
print(f"📈 Resultados: {DATA_OUTPUT}")


## 1. Funciones de Pre-procesamiento

### 1.1 Lectura de archivos MODFLOW


In [ ]:
def read_modflow_head_file(file_path, nrow, ncol):
    """
    Lee archivos de cabezas de MODFLOW (.hds)
    
    Parameters:
    -----------
    file_path : str
        Ruta al archivo .hds
    nrow, ncol : int
        Número de filas y columnas de la grilla
    
    Returns:
    --------
    numpy.ndarray
        Array 3D con las cabezas por estrato y tiempo
    """
    try:
        with open(file_path, 'rb') as f:
            # Leer cabecera del archivo
            header = np.fromfile(f, dtype=np.float32, count=1)
            data = np.fromfile(f, dtype=np.float32)
            
        # Reshape data según dimensiones
        nlay = len(data) // (nrow * ncol)
        data_reshaped = data.reshape(nlay, nrow, ncol)
        
        print(f"✅ Archivo {file_path} leído correctamente")
        print(f"   Dimensiones: {nlay} estratos, {nrow} filas, {ncol} columnas")
        
        return data_reshaped
        
    except Exception as e:
        print(f"❌ Error leyendo archivo {file_path}: {e}")
        return None

def read_observation_file(file_path):
    """
    Lee archivos de observaciones MODFLOW (.obs)
    
    Parameters:
    -----------
    file_path : str
        Ruta al archivo .obs
    
    Returns:
    --------
    pandas.DataFrame
        DataFrame con observaciones
    """
    try:
        # Leer archivo de observaciones
        obs_data = []
        
        with open(file_path, 'r') as f:
            lines = f.readlines()
            
        for line in lines:
            if line.strip() and not line.startswith('#'):
                parts = line.strip().split()
                if len(parts) >= 3:
                    obs_data.append({
                        'well_id': parts[0],
                        'time': float(parts[1]),
                        'head': float(parts[2])
                    })
        
        df = pd.DataFrame(obs_data)
        df['datetime'] = pd.to_datetime(df['time'], unit='D', origin='1900-01-01')
        
        print(f"✅ Archivo de observaciones {file_path} leído correctamente")
        print(f"   {len(df)} observaciones encontradas")
        
        return df
        
    except Exception as e:
        print(f"❌ Error leyendo archivo {file_path}: {e}")
        return None

# Ejemplo de uso
print("📋 Funciones de lectura de archivos MODFLOW definidas")


### 1.2 Limpieza y validación de datos


In [ ]:
def clean_observation_data(df, min_head=-1000, max_head=1000):
    """
    Limpia y valida datos de observaciones
    
    Parameters:
    -----------
    df : pandas.DataFrame
        DataFrame con observaciones
    min_head, max_head : float
        Valores mínimos y máximos válidos para cabezas
    
    Returns:
    --------
    pandas.DataFrame
        DataFrame limpio
    """
    print("🧹 Iniciando limpieza de datos de observaciones...")
    
    # Copiar DataFrame
    df_clean = df.copy()
    
    # Eliminar valores nulos
    initial_count = len(df_clean)
    df_clean = df_clean.dropna()
    print(f"   Eliminados {initial_count - len(df_clean)} registros con valores nulos")
    
    # Filtrar valores fuera de rango
    before_filter = len(df_clean)
    df_clean = df_clean[(df_clean['head'] >= min_head) & (df_clean['head'] <= max_head)]
    print(f"   Eliminados {before_filter - len(df_clean)} registros fuera de rango [{min_head}, {max_head}]")
    
    # Eliminar duplicados
    before_dedup = len(df_clean)
    df_clean = df_clean.drop_duplicates(subset=['well_id', 'time'])
    print(f"   Eliminados {before_dedup - len(df_clean)} registros duplicados")
    
    # Ordenar por pozo y tiempo
    df_clean = df_clean.sort_values(['well_id', 'time']).reset_index(drop=True)
    
    print(f"✅ Limpieza completada: {len(df_clean)} observaciones válidas")
    return df_clean

def validate_head_data(head_array, nodata_value=-999.99):
    """
    Valida datos de cabezas de MODFLOW
    
    Parameters:
    -----------
    head_array : numpy.ndarray
        Array 3D con cabezas
    nodata_value : float
        Valor que representa datos faltantes
    
    Returns:
    --------
    dict
        Estadísticas de validación
    """
    print("🔍 Validando datos de cabezas...")
    
    # Calcular estadísticas
    valid_data = head_array[head_array != nodata_value]
    
    stats = {
        'total_cells': head_array.size,
        'valid_cells': len(valid_data),
        'nodata_cells': head_array.size - len(valid_data),
        'min_head': np.min(valid_data) if len(valid_data) > 0 else None,
        'max_head': np.max(valid_data) if len(valid_data) > 0 else None,
        'mean_head': np.mean(valid_data) if len(valid_data) > 0 else None,
        'std_head': np.std(valid_data) if len(valid_data) > 0 else None
    }
    
    print(f"   Celdas totales: {stats['total_cells']}")
    print(f"   Celdas válidas: {stats['valid_cells']}")
    print(f"   Celdas sin datos: {stats['nodata_cells']}")
    print(f"   Rango de cabezas: {stats['min_head']:.2f} a {stats['max_head']:.2f}")
    
    return stats

print("📋 Funciones de limpieza y validación definidas")


## 2. Funciones de Post-procesamiento

### 2.1 Análisis estadístico


In [ ]:
def calculate_statistics(observed, simulated):
    """
    Calcula estadísticas de comparación entre datos observados y simulados
    
    Parameters:
    -----------
    observed : pandas.DataFrame
        Datos observados
    simulated : pandas.DataFrame
        Datos simulados
    
    Returns:
    --------
    dict
        Diccionario con estadísticas
    """
    print("📊 Calculando estadísticas de comparación...")
    
    # Merge de datos por pozo y tiempo
    merged = pd.merge(observed, simulated, on=['well_id', 'time'], 
                     suffixes=('_obs', '_sim'))
    
    # Calcular diferencias
    merged['residual'] = merged['head_obs'] - merged['head_sim']
    merged['abs_residual'] = np.abs(merged['residual'])
    merged['squared_residual'] = merged['residual']**2
    
    # Estadísticas globales
    stats = {
        'n_observations': len(merged),
        'mean_obs': merged['head_obs'].mean(),
        'mean_sim': merged['head_sim'].mean(),
        'std_obs': merged['head_obs'].std(),
        'std_sim': merged['head_sim'].std(),
        'mean_residual': merged['residual'].mean(),
        'std_residual': merged['residual'].std(),
        'mae': merged['abs_residual'].mean(),  # Mean Absolute Error
        'rmse': np.sqrt(merged['squared_residual'].mean()),  # Root Mean Square Error
        'r2': 1 - (merged['squared_residual'].sum() / 
                  ((merged['head_obs'] - merged['head_obs'].mean())**2).sum()),
        'nse': 1 - (merged['squared_residual'].sum() / 
                   ((merged['head_obs'] - merged['head_obs'].mean())**2).sum())  # Nash-Sutcliffe Efficiency
    }
    
    print(f"   Observaciones: {stats['n_observations']}")
    print(f"   MAE: {stats['mae']:.3f}")
    print(f"   RMSE: {stats['rmse']:.3f}")
    print(f"   R²: {stats['r2']:.3f}")
    print(f"   NSE: {stats['nse']:.3f}")
    
    return stats, merged

def analyze_by_well(merged_data):
    """
    Analiza estadísticas por pozo individual
    
    Parameters:
    -----------
    merged_data : pandas.DataFrame
        Datos fusionados observados-simulados
    
    Returns:
    --------
    pandas.DataFrame
        Estadísticas por pozo
    """
    print("🔍 Analizando estadísticas por pozo...")
    
    well_stats = []
    
    for well_id in merged_data['well_id'].unique():
        well_data = merged_data[merged_data['well_id'] == well_id]
        
        if len(well_data) > 1:  # Necesitamos al menos 2 puntos para estadísticas
            stats = {
                'well_id': well_id,
                'n_obs': len(well_data),
                'mae': well_data['abs_residual'].mean(),
                'rmse': np.sqrt(well_data['squared_residual'].mean()),
                'r2': 1 - (well_data['squared_residual'].sum() / 
                          ((well_data['head_obs'] - well_data['head_obs'].mean())**2).sum()),
                'mean_residual': well_data['residual'].mean(),
                'std_residual': well_data['residual'].std()
            }
            well_stats.append(stats)
    
    well_df = pd.DataFrame(well_stats)
    print(f"   Analizados {len(well_df)} pozos")
    
    return well_df

print("📋 Funciones de análisis estadístico definidas")


### 2.2 Visualizaciones


In [ ]:
def plot_time_series(merged_data, well_ids=None, save_path=None):
    """
    Genera gráficos de series temporales para pozos específicos
    
    Parameters:
    -----------
    merged_data : pandas.DataFrame
        Datos fusionados observados-simulados
    well_ids : list, optional
        Lista de IDs de pozos a graficar
    save_path : str, optional
        Ruta para guardar el gráfico
    """
    print("📈 Generando gráficos de series temporales...")
    
    if well_ids is None:
        well_ids = merged_data['well_id'].unique()[:5]  # Primeros 5 pozos por defecto
    
    n_wells = len(well_ids)
    fig, axes = plt.subplots(n_wells, 1, figsize=(12, 3*n_wells))
    
    if n_wells == 1:
        axes = [axes]
    
    for i, well_id in enumerate(well_ids):
        well_data = merged_data[merged_data['well_id'] == well_id].sort_values('time')
        
        axes[i].plot(well_data['time'], well_data['head_obs'], 'o-', 
                    label='Observado', color='blue', alpha=0.7)
        axes[i].plot(well_data['time'], well_data['head_sim'], 's-', 
                    label='Simulado', color='red', alpha=0.7)
        
        axes[i].set_title(f'Pozo {well_id}')
        axes[i].set_xlabel('Tiempo (días)')
        axes[i].set_ylabel('Cabeza (m)')
        axes[i].legend()
        axes[i].grid(True, alpha=0.3)
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"   Gráfico guardado en: {save_path}")
    
    plt.show()

def plot_scatter_comparison(merged_data, save_path=None):
    """
    Genera gráfico de dispersión observado vs simulado
    
    Parameters:
    -----------
    merged_data : pandas.DataFrame
        Datos fusionados observados-simulados
    save_path : str, optional
        Ruta para guardar el gráfico
    """
    print("📊 Generando gráfico de dispersión...")
    
    fig, ax = plt.subplots(1, 1, figsize=(8, 8))
    
    # Scatter plot
    ax.scatter(merged_data['head_obs'], merged_data['head_sim'], 
              alpha=0.6, s=50, color='blue')
    
    # Línea 1:1
    min_val = min(merged_data['head_obs'].min(), merged_data['head_sim'].min())
    max_val = max(merged_data['head_obs'].max(), merged_data['head_sim'].max())
    ax.plot([min_val, max_val], [min_val, max_val], 'r--', 
           label='Línea 1:1', linewidth=2)
    
    # Línea de regresión
    z = np.polyfit(merged_data['head_obs'], merged_data['head_sim'], 1)
    p = np.poly1d(z)
    ax.plot(merged_data['head_obs'], p(merged_data['head_obs']), 
           'g-', label=f'Regresión (R²={z[0]:.3f})', linewidth=2)
    
    ax.set_xlabel('Cabeza Observada (m)')
    ax.set_ylabel('Cabeza Simulada (m)')
    ax.set_title('Comparación Observado vs Simulado')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"   Gráfico guardado en: {save_path}")
    
    plt.show()

def plot_residuals(merged_data, save_path=None):
    """
    Genera gráficos de análisis de residuos
    
    Parameters:
    -----------
    merged_data : pandas.DataFrame
        Datos fusionados observados-simulados
    save_path : str, optional
        Ruta para guardar el gráfico
    """
    print("📉 Generando análisis de residuos...")
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # 1. Residuos vs tiempo
    axes[0,0].scatter(merged_data['time'], merged_data['residual'], alpha=0.6)
    axes[0,0].axhline(y=0, color='r', linestyle='--')
    axes[0,0].set_xlabel('Tiempo (días)')
    axes[0,0].set_ylabel('Residuo (m)')
    axes[0,0].set_title('Residuos vs Tiempo')
    axes[0,0].grid(True, alpha=0.3)
    
    # 2. Residuos vs valores observados
    axes[0,1].scatter(merged_data['head_obs'], merged_data['residual'], alpha=0.6)
    axes[0,1].axhline(y=0, color='r', linestyle='--')
    axes[0,1].set_xlabel('Cabeza Observada (m)')
    axes[0,1].set_ylabel('Residuo (m)')
    axes[0,1].set_title('Residuos vs Valores Observados')
    axes[0,1].grid(True, alpha=0.3)
    
    # 3. Histograma de residuos
    axes[1,0].hist(merged_data['residual'], bins=30, alpha=0.7, color='skyblue', edgecolor='black')
    axes[1,0].axvline(x=0, color='r', linestyle='--')
    axes[1,0].set_xlabel('Residuo (m)')
    axes[1,0].set_ylabel('Frecuencia')
    axes[1,0].set_title('Distribución de Residuos')
    axes[1,0].grid(True, alpha=0.3)
    
    # 4. Q-Q plot
    from scipy import stats
    stats.probplot(merged_data['residual'], dist="norm", plot=axes[1,1])
    axes[1,1].set_title('Q-Q Plot de Residuos')
    axes[1,1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"   Gráfico guardado en: {save_path}")
    
    plt.show()

print("📋 Funciones de visualización definidas")


## 3. Ejemplo de Uso con Datos Sintéticos

### 3.1 Generación de datos de ejemplo


In [ ]:
# Generar datos sintéticos para demostración
print("🔧 Generando datos sintéticos para demostración...")

# Parámetros del ejemplo
n_wells = 5
n_times = 100
time_start = 0
time_end = 365

# Generar datos observados (con ruido)
np.random.seed(42)  # Para reproducibilidad

observed_data = []
for well_id in range(1, n_wells + 1):
    # Tendencia base diferente para cada pozo
    base_head = 100 + well_id * 5  # Diferentes niveles base
    seasonal = 2 * np.sin(2 * np.pi * np.linspace(0, 1, n_times))  # Variación estacional
    trend = 0.1 * np.linspace(0, 1, n_times)  # Tendencia temporal
    noise = np.random.normal(0, 0.5, n_times)  # Ruido aleatorio
    
    times = np.linspace(time_start, time_end, n_times)
    heads = base_head + seasonal + trend + noise
    
    for t, h in zip(times, heads):
        observed_data.append({
            'well_id': f'PZ{well_id:02d}',
            'time': t,
            'head': h
        })

observed_df = pd.DataFrame(observed_data)

# Generar datos simulados (con algunos errores sistemáticos)
simulated_data = []
for well_id in range(1, n_wells + 1):
    well_obs = observed_df[observed_df['well_id'] == f'PZ{well_id:02d}']
    
    for _, row in well_obs.iterrows():
        # Simular error sistemático y aleatorio
        systematic_error = 0.5 if well_id % 2 == 0 else -0.3  # Error sistemático por pozo
        random_error = np.random.normal(0, 0.3)
        simulated_head = row['head'] + systematic_error + random_error
        
        simulated_data.append({
            'well_id': row['well_id'],
            'time': row['time'],
            'head': simulated_head
        })

simulated_df = pd.DataFrame(simulated_data)

print(f"✅ Datos sintéticos generados:")
print(f"   Pozos: {n_wells}")
print(f"   Observaciones por pozo: {n_times}")
print(f"   Total observaciones: {len(observed_df)}")
print(f"   Rango de cabezas observadas: {observed_df['head'].min():.2f} - {observed_df['head'].max():.2f} m")
print(f"   Rango de cabezas simuladas: {simulated_df['head'].min():.2f} - {simulated_df['head'].max():.2f} m")


### 3.2 Aplicación del flujo de trabajo completo


In [ ]:
# PASO 1: Limpiar datos observados
print("=" * 60)
print("PASO 1: PRE-PROCESAMIENTO")
print("=" * 60)

observed_clean = clean_observation_data(observed_df)
simulated_clean = clean_observation_data(simulated_df)

# PASO 2: Calcular estadísticas
print("\n" + "=" * 60)
print("PASO 2: ANÁLISIS ESTADÍSTICO")
print("=" * 60)

stats, merged_data = calculate_statistics(observed_clean, simulated_clean)
well_stats = analyze_by_well(merged_data)

# Mostrar estadísticas por pozo
print("\n📊 Estadísticas por pozo:")
print(well_stats.round(3))


In [ ]:
# PASO 3: Generar visualizaciones
print("\n" + "=" * 60)
print("PASO 3: VISUALIZACIONES")
print("=" * 60)

# Gráfico de series temporales
plot_time_series(merged_data, well_ids=['PZ01', 'PZ02', 'PZ03'], 
                save_path=FIGURES_DIR / "time_series.png")

# Gráfico de dispersión
plot_scatter_comparison(merged_data, 
                       save_path=FIGURES_DIR / "scatter_comparison.png")

# Análisis de residuos
plot_residuals(merged_data, 
              save_path=FIGURES_DIR / "residual_analysis.png")


In [ ]:
# PASO 4: Guardar resultados
print("\n" + "=" * 60)
print("PASO 4: GUARDAR RESULTADOS")
print("=" * 60)

# Guardar datos procesados
observed_clean.to_csv(DATA_PROCESSED / "observed_clean.csv", index=False)
simulated_clean.to_csv(DATA_PROCESSED / "simulated_clean.csv", index=False)
merged_data.to_csv(DATA_OUTPUT / "merged_data.csv", index=False)
well_stats.to_csv(DATA_OUTPUT / "well_statistics.csv", index=False)

# Guardar estadísticas globales
stats_df = pd.DataFrame([stats])
stats_df.to_csv(DATA_OUTPUT / "global_statistics.csv", index=False)

print("✅ Archivos guardados:")
print(f"   Datos observados limpios: {DATA_PROCESSED / 'observed_clean.csv'}")
print(f"   Datos simulados limpios: {DATA_PROCESSED / 'simulated_clean.csv'}")
print(f"   Datos fusionados: {DATA_OUTPUT / 'merged_data.csv'}")
print(f"   Estadísticas por pozo: {DATA_OUTPUT / 'well_statistics.csv'}")
print(f"   Estadísticas globales: {DATA_OUTPUT / 'global_statistics.csv'}")
print(f"   Gráficos: {FIGURES_DIR}")

# Resumen final
print("\n" + "=" * 60)
print("RESUMEN FINAL")
print("=" * 60)
print(f"📊 Total de observaciones analizadas: {stats['n_observations']}")
print(f"📈 Error absoluto medio (MAE): {stats['mae']:.3f} m")
print(f"📉 Error cuadrático medio (RMSE): {stats['rmse']:.3f} m")
print(f"🎯 Coeficiente de determinación (R²): {stats['r2']:.3f}")
print(f"⚡ Eficiencia Nash-Sutcliffe (NSE): {stats['nse']:.3f}")

# Interpretación de resultados
print("\n📋 Interpretación de resultados:")
if stats['nse'] > 0.7:
    print("   ✅ Excelente ajuste del modelo (NSE > 0.7)")
elif stats['nse'] > 0.5:
    print("   ⚠️  Ajuste aceptable del modelo (NSE > 0.5)")
else:
    print("   ❌ Ajuste deficiente del modelo (NSE < 0.5)")

if stats['r2'] > 0.8:
    print("   ✅ Alta correlación entre observado y simulado (R² > 0.8)")
elif stats['r2'] > 0.6:
    print("   ⚠️  Correlación moderada (R² > 0.6)")
else:
    print("   ❌ Baja correlación (R² < 0.6)")


## 4. Funciones Auxiliares para Archivos MODFLOW

### 4.1 Procesamiento de archivos .hds


In [ ]:
def extract_head_at_wells(head_array, well_locations, nrow, ncol):
    """
    Extrae valores de cabeza en ubicaciones específicas de pozos
    
    Parameters:
    -----------
    head_array : numpy.ndarray
        Array 3D con cabezas
    well_locations : list
        Lista de tuplas (row, col) para cada pozo
    nrow, ncol : int
        Dimensiones de la grilla
    
    Returns:
    --------
    pandas.DataFrame
        DataFrame con cabezas por pozo y estrato
    """
    print("🔍 Extrayendo cabezas en ubicaciones de pozos...")
    
    well_data = []
    
    for i, (row, col) in enumerate(well_locations):
        if 0 <= row < nrow and 0 <= col < ncol:
            for layer in range(head_array.shape[0]):
                head_value = head_array[layer, row, col]
                well_data.append({
                    'well_id': f'PZ{i+1:02d}',
                    'layer': layer + 1,
                    'row': row + 1,  # Convertir a índices 1-based
                    'col': col + 1,
                    'head': head_value
                })
        else:
            print(f"   ⚠️  Ubicación de pozo {i+1} fuera de rango: ({row}, {col})")
    
    df = pd.DataFrame(well_data)
    print(f"   ✅ Extraídas {len(df)} cabezas de {len(well_locations)} pozos")
    
    return df

def create_head_contour_plot(head_array, layer=0, save_path=None):
    """
    Crea mapa de contornos de cabezas
    
    Parameters:
    -----------
    head_array : numpy.ndarray
        Array 3D con cabezas
    layer : int
        Estrato a visualizar
    save_path : str, optional
        Ruta para guardar el gráfico
    """
    print(f"🗺️  Generando mapa de contornos para estrato {layer+1}...")
    
    fig, ax = plt.subplots(1, 1, figsize=(10, 8))
    
    # Crear contornos
    contour = ax.contour(head_array[layer], levels=20, colors='black', alpha=0.6)
    contour_filled = ax.contourf(head_array[layer], levels=20, cmap='viridis', alpha=0.8)
    
    # Añadir barra de color
    cbar = plt.colorbar(contour_filled, ax=ax)
    cbar.set_label('Cabeza (m)', rotation=270, labelpad=20)
    
    ax.set_title(f'Mapa de Contornos de Cabezas - Estrato {layer+1}')
    ax.set_xlabel('Columna')
    ax.set_ylabel('Fila')
    
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"   Mapa guardado en: {save_path}")
    
    plt.show()

print("📋 Funciones auxiliares para archivos MODFLOW definidas")


## 5. Flujo de Trabajo Completo

### 5.1 Función principal de procesamiento


In [ ]:
def process_modflow_data(observed_file=None, simulated_file=None, head_file=None, 
                        well_locations=None, nrow=None, ncol=None, 
                        output_dir=None, project_name="modflow_analysis"):
    """
    Función principal para procesar datos de MODFLOW
    
    Parameters:
    -----------
    observed_file : str, optional
        Ruta al archivo de observaciones
    simulated_file : str, optional
        Ruta al archivo de datos simulados
    head_file : str, optional
        Ruta al archivo .hds
    well_locations : list, optional
        Ubicaciones de pozos [(row, col), ...]
    nrow, ncol : int, optional
        Dimensiones de la grilla
    output_dir : str, optional
        Directorio de salida
    project_name : str
        Nombre del proyecto
    
    Returns:
    --------
    dict
        Diccionario con resultados del análisis
    """
    print("🚀 Iniciando procesamiento completo de datos MODFLOW")
    print("=" * 60)
    
    # Configurar directorios
    if output_dir is None:
        output_dir = Path.cwd() / "output" / project_name
    
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)
    
    results = {
        'project_name': project_name,
        'output_dir': str(output_dir),
        'processed_files': [],
        'statistics': {},
        'figures': []
    }
    
    # PASO 1: Procesar datos observados
    if observed_file and Path(observed_file).exists():
        print("📊 Procesando datos observados...")
        observed_df = read_observation_file(observed_file)
        if observed_df is not None:
            observed_clean = clean_observation_data(observed_df)
            observed_clean.to_csv(output_dir / "observed_clean.csv", index=False)
            results['processed_files'].append("observed_clean.csv")
            print("   ✅ Datos observados procesados")
    
    # PASO 2: Procesar datos simulados
    if simulated_file and Path(simulated_file).exists():
        print("🔧 Procesando datos simulados...")
        simulated_df = read_observation_file(simulated_file)
        if simulated_df is not None:
            simulated_clean = clean_observation_data(simulated_df)
            simulated_clean.to_csv(output_dir / "simulated_clean.csv", index=False)
            results['processed_files'].append("simulated_clean.csv")
            print("   ✅ Datos simulados procesados")
    
    # PASO 3: Procesar archivos de cabezas
    if head_file and Path(head_file).exists() and nrow and ncol:
        print("🗂️  Procesando archivo de cabezas...")
        head_array = read_modflow_head_file(head_file, nrow, ncol)
        if head_array is not None:
            # Validar datos
            head_stats = validate_head_data(head_array)
            results['head_statistics'] = head_stats
            
            # Extraer cabezas en pozos si se proporcionan ubicaciones
            if well_locations:
                well_heads = extract_head_at_wells(head_array, well_locations, nrow, ncol)
                well_heads.to_csv(output_dir / "well_heads.csv", index=False)
                results['processed_files'].append("well_heads.csv")
            
            # Crear mapa de contornos
            create_head_contour_plot(head_array, layer=0, 
                                   save_path=output_dir / "head_contour.png")
            results['figures'].append("head_contour.png")
            print("   ✅ Archivo de cabezas procesado")
    
    # PASO 4: Análisis estadístico (si tenemos datos observados y simulados)
    if 'observed_clean' in locals() and 'simulated_clean' in locals():
        print("📈 Realizando análisis estadístico...")
        stats, merged_data = calculate_statistics(observed_clean, simulated_clean)
        well_stats = analyze_by_well(merged_data)
        
        # Guardar resultados
        merged_data.to_csv(output_dir / "merged_data.csv", index=False)
        well_stats.to_csv(output_dir / "well_statistics.csv", index=False)
        stats_df = pd.DataFrame([stats])
        stats_df.to_csv(output_dir / "global_statistics.csv", index=False)
        
        results['statistics'] = stats
        results['processed_files'].extend(["merged_data.csv", "well_statistics.csv", "global_statistics.csv"])
        
        # Generar visualizaciones
        print("📊 Generando visualizaciones...")
        plot_time_series(merged_data, save_path=output_dir / "time_series.png")
        plot_scatter_comparison(merged_data, save_path=output_dir / "scatter_comparison.png")
        plot_residuals(merged_data, save_path=output_dir / "residual_analysis.png")
        
        results['figures'].extend(["time_series.png", "scatter_comparison.png", "residual_analysis.png"])
        print("   ✅ Análisis estadístico completado")
    
    # PASO 5: Generar reporte
    print("📝 Generando reporte...")
    generate_report(results, output_dir / "analysis_report.txt")
    results['processed_files'].append("analysis_report.txt")
    
    print("\n" + "=" * 60)
    print("✅ PROCESAMIENTO COMPLETADO")
    print("=" * 60)
    print(f"📁 Directorio de salida: {output_dir}")
    print(f"📊 Archivos procesados: {len(results['processed_files'])}")
    print(f"📈 Gráficos generados: {len(results['figures'])}")
    
    return results

def generate_report(results, report_path):
    """
    Genera un reporte de texto con los resultados del análisis
    
    Parameters:
    -----------
    results : dict
        Resultados del análisis
    report_path : str
        Ruta para guardar el reporte
    """
    with open(report_path, 'w', encoding='utf-8') as f:
        f.write("REPORTE DE ANÁLISIS MODFLOW\n")
        f.write("=" * 50 + "\n\n")
        f.write(f"Proyecto: {results['project_name']}\n")
        f.write(f"Fecha: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
        
        f.write("ARCHIVOS PROCESADOS:\n")
        f.write("-" * 30 + "\n")
        for file in results['processed_files']:
            f.write(f"• {file}\n")
        
        f.write("\nGRÁFICOS GENERADOS:\n")
        f.write("-" * 30 + "\n")
        for fig in results['figures']:
            f.write(f"• {fig}\n")
        
        if 'statistics' in results and results['statistics']:
            f.write("\nESTADÍSTICAS GLOBALES:\n")
            f.write("-" * 30 + "\n")
            stats = results['statistics']
            f.write(f"Observaciones: {stats.get('n_observations', 'N/A')}\n")
            f.write(f"MAE: {stats.get('mae', 'N/A'):.3f} m\n")
            f.write(f"RMSE: {stats.get('rmse', 'N/A'):.3f} m\n")
            f.write(f"R²: {stats.get('r2', 'N/A'):.3f}\n")
            f.write(f"NSE: {stats.get('nse', 'N/A'):.3f}\n")
    
    print(f"   📄 Reporte guardado en: {report_path}")

print("📋 Función principal de procesamiento definida")


## 6. Instrucciones de Uso

### 6.1 Para usar con datos reales:

```python
# Ejemplo de uso con archivos reales
results = process_modflow_data(
    observed_file="data/raw/observations.obs",
    simulated_file="data/raw/simulated.obs", 
    head_file="data/raw/model.hds",
    well_locations=[(10, 15), (20, 25), (30, 35)],  # (fila, columna)
    nrow=50, ncol=60,
    output_dir="results/analysis_2024",
    project_name="mi_proyecto_modflow"
)
```

### 6.2 Estructura de archivos de entrada:

**Archivo de observaciones (.obs):**
```
# Comentarios opcionales
PZ01 0.0 105.2
PZ01 30.0 104.8
PZ02 0.0 98.5
PZ02 30.0 98.1
```

**Archivo de cabezas (.hds):**
- Archivo binario generado por MODFLOW
- Requiere especificar dimensiones de la grilla (nrow, ncol)

### 6.3 Interpretación de resultados:

- **MAE (Mean Absolute Error)**: Error absoluto medio en metros
- **RMSE (Root Mean Square Error)**: Error cuadrático medio en metros  
- **R²**: Coeficiente de determinación (0-1, mayor es mejor)
- **NSE (Nash-Sutcliffe Efficiency)**: Eficiencia del modelo (-∞ a 1, mayor es mejor)

**Criterios de evaluación:**
- NSE > 0.7: Excelente ajuste
- NSE > 0.5: Ajuste aceptable  
- NSE < 0.5: Ajuste deficiente


## 7. Archivos de Configuración

### 7.1 requirements.txt
